## Task8文本数据
### 一、str对象
#### 1. str对象的设计意图

`str`对象是定义在`Index`或`Series`上的属性，专门用于逐元素处理文本内容，其内部定义了大量方法，因此对一个序列进行文本处理，首先需要获取其`str`对象。



In [1]:
%cd drive/MyDrive/

/content/drive/MyDrive


In [3]:
import pandas as pd
import numpy as np

In [4]:
# 字母转为大写
s = pd.Series(['abcd', 'efg', 'hi'])
s.str.upper()

0    ABCD
1     EFG
2      HI
dtype: object

#### 2. []索引器

对于`str`对象而言，可理解为其对字符串进行了序列化的操作。

In [7]:
# 通过[]可以取出某个位置的元素
s.str[:2]

0    ab
1    ef
2    hi
dtype: object

#### 3. string类型

绝大多数对于`object`和`string`类型的序列使用`str`对象方法产生的结果是一致的，但是有两点较大差异：
* 对于某些对象的 `str` 序列化方法不同：应当尽量保证每一个序列中的值都是字符串的情况下才使用 `str` 属性，但这并不是必须的，其必要条件是序列中至少有一个可迭代（`Iterable`）对象，包括但不限于字符串、字典、列表。对于一个可迭代对象， `string` 类型的 `str` 对象和 `object` 类型的 `str` 对象返回结果可能是不同的。

* `string` 类型是 `Nullable` 类型，但 `object` 不是。这意味着 `string` 类型的序列，如果调用的 `str` 方法返回值为整数 `Series` 和布尔`Series` 时，其分别对应的 `dtype` 是 `Int` 和 `boolean` 的 `Nullable` 类型，而 `object` 类型则会分别返回 `int/float` 和 `bool/object` ，取决于缺失值的存在与否。同时，字符串的比较操作，也具有相似的特性， `string` 返回 `Nullable` 类型，但 `object` 不会。

In [8]:
s = pd.Series([{1: 'temp_1', 2: 'temp_2'}, ['a', 'b'], 0.5, 'my_string'])

In [9]:
s.str[1]

0    temp_1
1         b
2       NaN
3         y
dtype: object

In [10]:
s.astype('string').str[1]

0    1
1    '
2    .
3    y
dtype: string

In [11]:
s = pd.Series(['a'])

In [12]:
s.str.len()

0    1
dtype: int64

In [13]:
s.astype('string').str.len()

0    1
dtype: Int64

In [14]:
s == 'a'

0    True
dtype: bool

In [15]:
s.astype('string') == 'a'

0    True
dtype: boolean

In [16]:
s = pd.Series(['a', np.nan]) # 带有缺失值
s.str.len()

0    1.0
1    NaN
dtype: float64

In [17]:
s.astype('string').str.len()

0       1
1    <NA>
dtype: Int64

In [18]:
s == 'a'

0     True
1    False
dtype: bool

In [19]:
s.astype('string') == 'a'

0    True
1    <NA>
dtype: boolean

> 最后需要注意的是，对于全体元素为数值类型的序列，即使其类型为 object 或者 category 也不允许直接使用 str 属性。如果需要把数字当成 string 类型处理，可以使用 astype 强制转换为 string 类型的 Series :

In [20]:
s = pd.Series([12, 345, 6789])
s.astype('string').str[1]

0    2
1    4
2    7
dtype: string

### 二、正则表达式基础
#### 1. 一般字符的匹配
正则表达式是一种按照某种正则模式，从左到右匹配字符串中内容的一种工具。对于一般的字符而言，它可以找到其所在的位置。


In [21]:
import re

In [23]:
# 在字符串中找出apple
# 第一个参数是正则表达式，第二个参数是待匹配的字符串
re.findall(r'Apple', 'Apple!This is an Apple!')

['Apple', 'Apple']

#### 2. 元字符基础


* `.` ：匹配除换行符以外的任意字符

* `[ ]`：字符类，匹配方括号中包含的任意字符。

* `[^ ]`：否定字符类，匹配方括号中不包含的任意字符

* `*`：匹配前面的子表达式零次或多次

* `+`：匹配前面的子表达式一次或多次

* `?`：匹配前面的子表达式零次或一次

* `{n,m}`：花括号，匹配前面字符至少 n 次，但是不超过 m 次

* `(xyz)`：字符组，按照确切的顺序匹配字符xyz。

* `|`：分支结构，匹配符号之前的字符或后面的字符

* `\`：转义符，它可以还原元字符原来的含义

* `^`：匹配行的开始

* `$`：匹配行的结束

In [24]:
# 匹配除换行符以外的任意字符
re.findall(r'.', 'abc')

['a', 'b', 'c']

In [25]:
# 匹配方括号中包含的任意字符。
re.findall(r'[ac]', 'abc')

['a', 'c']

In [26]:
# 匹配方括号中不包含的任意字符
re.findall(r'[^ac]', 'abc')

['b']

In [27]:

re.findall(r'[ab]{2}', 'aaaabbbb') # {n}指匹配n次

['aa', 'aa', 'bb', 'bb']

In [28]:
# 匹配符号之前的字符或后面的字符
re.findall(r'aaa|bbb', 'aaaabbbb')

['aaa', 'bbb']

In [29]:
re.findall(r'a\\?|a\*', 'aa?a*a')

['a', 'a', 'a', 'a']

In [30]:
re.findall(r'a?.', 'abaacadaae')



['ab', 'aa', 'c', 'ad', 'aa', 'e']

#### 3. 简写字符集
等价于一组字符的集合：

* `\w`：匹配所有字母、数字、下划线: `[a-zA-Z0-9_]`

* `\W`：匹配非字母和数字的字符: `[^\w]`

* `\d`：匹配数字: `[0-9]`

* `\D`：匹配非数字: `[^\d]`

* `\s`：匹配空格符: `[\t\n\f\r\p{Z}]`

* `\S`：匹配非空格符: `[^\s]`

* `\B`：匹配一组非空字符开头或结尾的位置，不代表具体字符

In [31]:
re.findall(r'.s', 'Apple! This Is an Apple!')

['is', 'Is']

In [32]:
re.findall(r'\w{2}', '09 8? 7w c_ 9q p@')

['09', '7w', 'c_', '9q']

In [33]:
re.findall(r'\w\W\B', '09 8? 7w c_ 9q p@')

['8?', 'p@']

In [34]:
re.findall(r'.\s.', 'Constant dropping wears the stone.')

['t d', 'g w', 's t', 'e s']

In [35]:
re.findall(r'上海市(.{2,3}区)(.{2,3}路)(\d+号)',
           '上海市黄浦区方浜中路249号 上海市宝山区密山路5号')

[('黄浦区', '方浜中路', '249号'), ('宝山区', '密山路', '5号')]

### 三、文本处理的五类操作
#### 1. 拆分
`str.split` 能够把字符串的列进行拆分，其中第一个参数为正则表达式，可选参数包括从左到右的最大拆分次数 `n` ，是否展开为多个列 `expand` 。

In [36]:
s = pd.Series(['上海市黄浦区方浜中路249号',
           '上海市宝山区密山路5号'])

In [37]:
s.str.split('[市区路]')

0    [上海, 黄浦, 方浜中, 249号]
1       [上海, 宝山, 密山, 5号]
dtype: object

In [38]:
s.str.split('[市区路]', n=2, expand=True)

,0,1,2
0,上海,黄浦,方浜中路249号
1,上海,宝山,密山路5号


#### 2. 合并
 `str.join` 和 `str.cat` 。 `str.join` 表示用某个连接符把 `Series` 中的字符串列表连接起来，如果列表中出现了非字符串元素则返回缺失值。

In [40]:
s = pd.Series([['a','b'], [1, 'a'], [['a', 'b'], 'c']])

In [41]:
s.str.join('-')

0    a-b
1    NaN
2    NaN
dtype: object

`str.cat` 用于合并两个序列，主要参数为连接符 `sep` 、连接形式 `join` 以及缺失值替代符号 `na_rep` ，其中连接形式默认为以索引为键的左连接。

In [42]:
s1 = pd.Series(['a','b'])

In [43]:
s2 = pd.Series(['cat','dog'])

In [44]:
s1.str.cat(s2,sep='-')

0    a-cat
1    b-dog
dtype: object

In [45]:
s2.index = [1, 2]

In [47]:
s2

1    cat
2    dog
dtype: object

In [48]:
s1

0    a
1    b
dtype: object

In [46]:
s1.str.cat(s2, sep='-', na_rep='?', join='outer')

0      a-?
1    b-cat
2    ?-dog
dtype: object

#### 3. 匹配
`str.contains` 返回了每个字符串是否包含正则模式的布尔序列。

In [49]:
s = pd.Series(['my cat', 'he is fat', 'railway station'])

In [50]:
s.str.contains('\s\wat')

0     True
1     True
2    False
dtype: bool

`str.startswith` 和 `str.endswith` 返回了每个字符串以给定模式为开始和结束的布尔序列，它们都不支持正则表达式。

In [52]:
s.str.startswith('my')

0     True
1    False
2    False
dtype: bool

In [53]:
s.str.endswith('t')

0     True
1     True
2    False
dtype: bool

如果需要用正则表达式来检测开始或结束字符串的模式，可以使用 `str.match` ，其返回了每个字符串起始处是否符合给定正则模式的布尔序列.

In [55]:
s.str.match('m|h')

0     True
1     True
2    False
dtype: bool

In [56]:
s.str[::-1].str.match('ta[f|g]|n') # 反转后匹配

0    False
1     True
2     True
dtype: bool

In [57]:
s.str.contains('^[m|h]')

0     True
1     True
2    False
dtype: bool

In [58]:
s.str.contains('[f|g]at|n$')

0    False
1     True
2     True
dtype: bool

返回索引的匹配函数: `str.find` 与 `str.rfind` ，其分别返回从左到右和从右到左第一次匹配的位置的索引，未找到则返回-1。需要注意的是这两个函数不支持正则匹配，只能用于字符子串的匹配

In [59]:
s = pd.Series(['This is an apple. That is not an apple.'])

In [60]:
s.str.find('apple')

0    11
dtype: int64

In [61]:
s.str.rfind('apple')

0    33
dtype: int64

#### 4. 替换
`str.replace` 和 `replace` 并不是一个函数，在使用字符串替换时应当使用前者。

In [62]:
s = pd.Series(['a_1_b','c_?'])
s.str.replace('\d|\?', 'new', regex=True)

0    a_new_b
1      c_new
dtype: object

当需要对不同部分进行有差别的替换时，可以利用 子组 的方法，并且此时可以通过传入自定义的替换函数来分别进行处理，注意 group(k) 代表匹配到的第 k 个子组（圆括号之间的内容）

In [63]:
s = pd.Series(['上海市黄浦区方浜中路249号',
           '上海市宝山区密山路5号',
           '北京市昌平区北农路2号'])

In [64]:
pat = '(\w+市)(\w+区)(\w+路)(\d+号)'

city = {'上海市': 'Shanghai', '北京市': 'Beijing'}

district = {'昌平区': 'CP District',
     '黄浦区': 'HP District',
     '宝山区': 'BS District'}

road = {'方浜中路': 'Mid Fangbin Road',
   '密山路': 'Mishan Road',
   '北农路': 'Beinong Road'}

In [65]:
def my_func(m):
    str_city = city[m.group(1)]
    str_district = district[m.group(2)]
    str_road = road[m.group(3)]
    str_no = 'No. ' + m.group(4)[:-1]
    return ' '.join([str_city,
        str_district,
        str_road,
        str_no])

In [67]:
s.str.replace(pat, my_func, regex=True)

0    Shanghai HP District Mid Fangbin Road No. 249
1           Shanghai BS District Mishan Road No. 5
2           Beijing CP District Beinong Road No. 2
dtype: object

In [68]:
# 使用 命名子组 更加清晰地写出子组代表的含义
pat = '(?P<市名>\w+市)(?P<区名>\w+区)(?P<路名>\w+路)(?P<编号>\d+号)'
def my_func(m):
    str_city = city[m.group('市名')]
    str_district = district[m.group('区名')]
    str_road = road[m.group('路名')]
    str_no = 'No. ' + m.group('编号')[:-1]
    return ' '.join([str_city,
        str_district,
        str_road,
        str_no])

In [69]:
s.str.replace(pat, my_func, regex=True)

0    Shanghai HP District Mid Fangbin Road No. 249
1           Shanghai BS District Mishan Road No. 5
2           Beijing CP District Beinong Road No. 2
dtype: object

#### 5. 提取

提取既可以认为是一种返回具体元素（而不是布尔值或元素对应的索引位置）的匹配操作，也可以认为是一种特殊的拆分操作。

In [70]:
pat = '(\w+市)(\w+区)(\w+路)(\d+号)'
s.str.extract(pat)

,0,1,2,3
0,上海市,黄浦区,方浜中路,249号
1,上海市,宝山区,密山路,5号
2,北京市,昌平区,北农路,2号


通过子组的命名，可以直接对新生成 DataFrame 的列命名：

In [71]:
pat = '(?P<市名>\w+市)(?P<区名>\w+区)(?P<路名>\w+路)(?P<编号>\d+号)'
s.str.extract(pat)

,市名,区名,路名,编号
0,上海市,黄浦区,方浜中路,249号
1,上海市,宝山区,密山路,5号
2,北京市,昌平区,北农路,2号


str.extractall 不同于 str.extract 只匹配一次，它会把所有符合条件的模式全部匹配出来，如果存在多个结果，则以多级索引的方式存储。

In [72]:
s = pd.Series(['A135T15,A26S5','B674S2,B25T6'], index = ['my_A','my_B'])
pat = '[A|B](\d+)[T|S](\d+)'
s.str.extractall(pat)

0   1
     match         
my_A 0      135  15
     1       26   5
my_B 0      674   2
     1       25   6

In [73]:
pat_with_name = '[A|B](?P<name1>\d+)[T|S](?P<name2>\d+)'

In [74]:
s.str.extractall(pat_with_name)

name1 name2
     match            
my_A 0       135    15
     1        26     5
my_B 0       674     2
     1        25     6

str.findall 的功能类似于 str.extractall ，区别在于前者把结果存入列表中，而后者处理为多级索引，每个行只对应一组匹配，而不是把所有匹配组合构成列表。

In [75]:
s.str.findall(pat)

my_A    [(135, 15), (26, 5)]
my_B     [(674, 2), (25, 6)]
dtype: object

### 四、常用字符串函数
#### 1. 字母型函数
upper, lower, title, capitalize, swapcase 这五个函数主要用于字母的大小写转化

In [76]:
s = pd.Series(['lower', 'CAPITALS', 'this is a sentence', 'SwApCaSe'])
s.str.upper()

0                 LOWER
1              CAPITALS
2    THIS IS A SENTENCE
3              SWAPCASE
dtype: object

In [77]:
s.str.lower()

0                 lower
1              capitals
2    this is a sentence
3              swapcase
dtype: object

In [78]:
s.str.title()

0                 Lower
1              Capitals
2    This Is A Sentence
3              Swapcase
dtype: object

In [79]:
s.str.capitalize()

0                 Lower
1              Capitals
2    This is a sentence
3              Swapcase
dtype: object

In [80]:
s.str.swapcase()

0                 LOWER
1              capitals
2    THIS IS A SENTENCE
3              sWaPcAsE
dtype: object

#### 2. 数值型函数
 pd.to_numeric 方法，它虽然不是 str 对象上的方法，但是能够对字符格式的数值进行快速转换和筛选。其主要参数包括 errors 和 downcast 分别代表了非数值的处理模式和转换类型。其中，对于不能转换为数值的有三种 errors 选项， raise, coerce, ignore 分别表示直接报错、设为缺失以及保持原来的字符串。

In [81]:
s = pd.Series(['1', '2.2', '2e', '??', '-2.1', '0'])

In [82]:
pd.to_numeric(s, errors='ignore')

0       1
1     2.2
2      2e
3      ??
4    -2.1
5       0
dtype: object

In [83]:
pd.to_numeric(s, errors='coerce')

0    1.0
1    2.2
2    NaN
3    NaN
4   -2.1
5    0.0
dtype: float64

在数据清洗时，可以利用 coerce 的设定，快速查看非数值型的行：

In [84]:
s[pd.to_numeric(s, errors='coerce').isna()]

2    2e
3    ??
dtype: object

#### 3. 统计型函数
count 和 len 的作用分别是返回出现正则模式的次数和字符串的长度

In [85]:
s = pd.Series(['cat rat fat at', 'get feed sheet heat'])

In [86]:
s.str.count('[r|f]at|ee')

0    2
1    2
dtype: int64

In [87]:
s.str.len()

0    14
1    19
dtype: int64

#### 4. 格式型函数
* 除空型
* 填充型。

其中，第一类函数一共有三种，它们分别是 strip, rstrip, lstrip ，分别代表去除两侧空格、右侧空格和左侧空格。这些函数在数据清洗时是有用的，特别是列名含有非法空格的时候。

In [88]:
my_index = pd.Index([' col1', 'col2 ', ' col3 '])

In [89]:
my_index.str.strip().str.len()

Int64Index([4, 4, 4], dtype='int64')

In [90]:
my_index.str.rstrip().str.len()

Int64Index([5, 4, 5], dtype='int64')

In [91]:
my_index.str.lstrip().str.len()

Int64Index([4, 5, 5], dtype='int64')

对于填充型函数而言， pad 是最灵活的，它可以选定字符串长度、填充的方向和填充内容。

In [93]:
s = pd.Series(['a','b','c'])

In [94]:
s.str.pad(5,'left','*')

0    ****a
1    ****b
2    ****c
dtype: object

In [96]:
s.str.pad(5,'right','*')

0    a****
1    b****
2    c****
dtype: object

In [97]:
s.str.pad(5,'both','*')

0    **a**
1    **b**
2    **c**
dtype: object

上述的三种情况可以分别用 rjust, ljust, center 来等效完成，需要注意 ljust 是指右侧填充而不是左侧填充

In [98]:
s.str.rjust(5, '*')

0    ****a
1    ****b
2    ****c
dtype: object

In [99]:
s.str.ljust(5, '*')

0    a****
1    b****
2    c****
dtype: object

In [100]:
s.str.center(5, '*')

0    **a**
1    **b**
2    **c**
dtype: object

在读取 excel 文件时，经常会出现数字前补0的需求，例如证券代码读入的时候会把”000007”作为数值7来处理， pandas 中除了可以使用上面的左侧填充函数进行操作之外，还可用 zfill 来实现

In [101]:
s = pd.Series([7, 155, 303000]).astype('string')

In [102]:
s.str.pad(6,'left','0')

0    000007
1    000155
2    303000
dtype: string

In [103]:
s.str.rjust(6,'0')

0    000007
1    000155
2    303000
dtype: string

In [104]:
s.str.zfill(6)

0    000007
1    000155
2    303000
dtype: string

### 五、练习
#### Ex1：房屋信息数据集
现有一份房屋信息数据集如下：
```python
In [114]: df = pd.read_excel('data/house_info.xls', usecols=[
   .....:                 'floor','year','area','price'])
   .....: 

In [115]: df.head(3)
Out[115]: 
      floor    year    area price
0   高层（共6层）  1986年建  58.23㎡  155万
1  中层（共20层）  2020年建     88㎡  155万
2  低层（共28层）  2010年建  89.33㎡  365万
```
1. 将 year 列改为整数年份存储。

2. 将 floor 列替换为 Level, Highest 两列，其中的元素分别为 string 类型的层类别（高层、中层、低层）与整数类型的最高层数。

3. 计算房屋每平米的均价 avg_price ，以 \*** 元/平米 的格式存储到表中，其中 \*** 为整数。

In [105]:
df = pd.read_excel('data/house_info.xls', usecols=[
            'floor','year','area','price'])

In [107]:
df.head()

,floor,year,area,price
0,高层（共6层）,1986年建,58.23㎡,155万
1,中层（共20层）,2020年建,88㎡,155万
2,低层（共28层）,2010年建,89.33㎡,365万
3,低层（共20层）,2014年建,82㎡,308万
4,高层（共1层）,2015年建,98㎡,117万


In [108]:
# 1. 将 year 列改为整数年份存储
df.year = pd.to_numeric(df.year.str[:-2]).astype('Int64')
df.head()

,floor,year,area,price
0,高层（共6层）,1986,58.23㎡,155万
1,中层（共20层）,2020,88㎡,155万
2,低层（共28层）,2010,89.33㎡,365万
3,低层（共20层）,2014,82㎡,308万
4,高层（共1层）,2015,98㎡,117万


In [109]:
# 将 floor 列替换为 Level, Highest 两列，
# 其中的元素分别为 string 类型的层类别（高层、中层、低层）与整数类型的最高层数。
pat = '(\w层)（共(\d+)层）'

new_cols = df.floor.str.extract(pat).rename(
            columns={0:'Level', 1:'Highest'})

df = pd.concat([df.drop(columns=['floor']), new_cols], 1)
df.head()

,year,area,price,Level,Highest
0,1986,58.23㎡,155万,高层,6
1,2020,88㎡,155万,中层,20
2,2010,89.33㎡,365万,低层,28
3,2014,82㎡,308万,低层,20
4,2015,98㎡,117万,高层,1


In [110]:
# 3. 计算房屋每平米的均价 avg_price ，
# 以 \***元/平米 的格式存储到表中，其中 \*** 为整数。
s_area = pd.to_numeric(df.area.str[:-1])
s_price = pd.to_numeric(df.price.str[:-1])
df['avg_price'] = ((s_price/s_area)*10000).astype(
            'int').astype('string') + '元/平米'
df.head()

,year,area,price,Level,Highest,avg_price
0,1986,58.23㎡,155万,高层,6,26618元/平米
1,2020,88㎡,155万,中层,20,17613元/平米
2,2010,89.33㎡,365万,低层,28,40859元/平米
3,2014,82㎡,308万,低层,20,37560元/平米
4,2015,98㎡,117万,高层,1,11938元/平米


#### Ex2：《权力的游戏》剧本数据集
现有一份权力的游戏剧本数据集如下：
```python
In [116]: df = pd.read_csv('data/script.csv')

In [117]: df.head(3)
Out[117]: 
  Release Date    Season   Episode      Episode Title          Name                                           Sentence
0   2011-04-17  Season 1  Episode 1  Winter is Coming  waymar royce  What do you expect? They're savages. One lot s...
1   2011-04-17  Season 1  Episode 1  Winter is Coming          will  I've never seen wildlings do a thing like this...
2   2011-04-17  Season 1  Episode 1  Winter is Coming  waymar royce                             How close did you get?
```
1. 计算每一个 Episode 的台词条数。

2. 以空格为单词的分割符号，请求出单句台词平均单词量最多的前五个人。

3. 若某人的台词中含有问号，那么下一个说台词的人即为回答者。若上一人台词中含有 n 个问号，则认为回答者回答了 n 个问题，请求出回答最多问题的前五个人。

In [111]:
# 计算每一个 Episode 的台词条数。
df = pd.read_csv('data/script.csv')
df.columns = df.columns.str.strip()
df.groupby(['Season', 'Episode'])['Sentence'].count().head()

Season    Episode   
Season 1  Episode 1     327
          Episode 10    266
          Episode 2     283
          Episode 3     353
          Episode 4     404
Name: Sentence, dtype: int64

In [112]:
# 以空格为单词的分割符号，请求出单句台词平均单词量最多的前五个人。
df.set_index('Name').Sentence.str.split().str.len(
      ).groupby('Name').mean().sort_values(ascending=False).head()

Name
male singer          109.000000
slave owner           77.000000
manderly              62.000000
lollys stokeworth     62.000000
dothraki matron       56.666667
Name: Sentence, dtype: float64

In [113]:
# 若某人的台词中含有问号，那么下一个说台词的人即为回答者。
#若上一人台词中含有 n 个问号，则认为回答者回答了 n 个问题，
#请求出回答最多问题的前五个人。
s = pd.Series(df.Sentence.values, index=df.Name.shift(-1))
s.str.count('\?').groupby('Name').sum().sort_values(ascending=False).head()

Name
tyrion lannister    527
jon snow            374
jaime lannister     283
arya stark          265
cersei lannister    246
dtype: int64